# Work Done to Compress Gas

**Rio Agustian Gilang Fernando**

**Advanced Material Science and Nanotechnology**

*Question*

How much work is done in compressing gaseous nitrogen, initially at 1 atm, isothermally at 298 K to 200 bar into 15 L scuba diving cylinder? Take $N_2$ to obey the van der Waals equation of state formula,

$$
p = \frac{nRT}{V-nb} - \frac{n^2a}{V^2}
$$

with pramters $a = 1.370 L^2 bar mol^{-2}$ and $b = 0.0387 L mol^{-1}$.

Obtain $V_1$ and then use the following expression for the work done:

$$
w = -\int_{V_1}^{V_2} p \mathrm{d}V,
$$

to compare the value for $w$ obtained by analytical integration and by with numerical integration using ```scipy.integrate.quad```. Also, compare the result with the same analysis carried out under the assumption that $N_2$ behaves as an ideal gas.

*Solution*

If the process is isothermal then the temperature T = 298 K is constant. Let the initial pressure by p1 = 101325 Pa and the volume V1 (to be determined). The final pressure and volume are $p_2 = 2 \times 10^7$ Pa and $V_2 = 15 L = 1.5 \times 10^{-2} m^3$.

In SI units, the van der Waals parameters are $a = 0.137 m^6 Pa mol^{-2}$ and $b = 3.87 \times 10^{-5} m^3 mol^{-1}$.

In [16]:
import numpy as np
from numpy.polynomial import Polynomial
from scipy.integrate import quad

# Define some quantities
R = 8.314 # Gas constant, J.K-1.mol-1
T = 298 # Temperature (K)2
p1 = 101325 # Initial pressure (Pa)
p2, V2 = 2.e7, 1.5e-2 # Final pressure (Pa) and volume (m3)
a, b = 0.137, 3.87e-5 # van der Waals parameters (m6.Pa.mol-2 and m3.mol-1)

First, we need to know $n$, the number of moles of gas compressed, and hence $V_1$, the initial volume of the gas. Rearranging the van der Waals equation gives:

\begin{align}
&pV^2 (V-nb) = nRTV^2 - n^2a(V-b) \\
\Rightarrow &abn^3 - aVn^2 + V^2 (bp + RT)n-pV^3=0
\end{align}

This is a cubic equation in n which could be solved analytically, but is easier to handle numerically:

In [18]:
# A Numpy Polynomial representing the cubic equation in n.
poly = Polynomial([-p2 * V2**3, V2**2 * (b*p2 + R*T), -a * V2, a * b])
roots = poly.roots()
roots

array([120.29497623  +0.j        , 133.6509615 -296.59957559j,
       133.6509615 +296.59957559j])

Of the three roots, only one is real so we are still in the region of phase space where the van der Waals equation is valid. Extracting this value and explicitly dropping its (zero) imaginary part gives n:

In [22]:
np.isreal(roots)

array([ True, False, False])

In [30]:
n = roots[np.isreal(roots)][0].real
n

120.29497623236371

Comparing this with the value giveb by assuming ideal gas behavior, it can be seen that there is a small difference:

In [33]:
nideal = p2 * V2 / R / T
nideal

121.08628931873625

The volume of this amount of nitrogen gas at 1 atm pressure can be approximated well by the ideal gas law, however:

In [36]:
V1 = n * R * T / p1
V1

2.9414208226397216

The work integral can be evaluated analytically for a van der Waals gas,

\begin{align}
w &= - \int_{V_1}^{V_2} p \mathrm{d}V \\
&=-\int_{V_1}^{V_2} \frac{nRT}{V-nb} - \frac{n^2a}{V^2} \mathrm{d}V \\
&=-nRT \ln{\left(\frac{V_2 -nb}{V_1-nb}\right)} - n^2 a \left(\frac{1}{V_2}-\frac{1}{V_1}\right).
\end{align}

In [45]:
term1 = -n*R*T*np.log((V2 - n*b)/(V1 - n*b))
term2 = -n**2* a * (1/V2 - 1/V1)
work = term1 + term2
work

1552012.6088384087

This is about 1.55 MJ. For completeness, compare with the work required assuming the gas to be ideal:

In [48]:
work_ideal = -n*R*T*np.log(V2 /V1)
work_ideal

1573230.4681104878

or 1.57 MJ: a little grater because an ideal gas has no attractive foreces between its molecules.

To take the numerical approach:

In [55]:
def p(V, n, R, T, a, b):
    return n * R * T / (V - n * b) - n**2 * a / V**2

wp, err = quad(p, V1, V2, args=(n, R, T, a, b))
print(-wp, err)

1552012.6088384101 0.004105310200539947


Unsuprisingly, this is close to the analytical value found above.